In [8]:
import os
from fastai.vision import *
from fastai.callbacks import *
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import warnings
warnings.filterwarnings('ignore')

In [9]:
# to set device
# os.environ["CUDA_VISIBLE_DEVICES"]="1"
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
# torch.cuda.current_device()

In [10]:
# path for image data
path = Path("path of folder containing image data")

In [12]:
# Image Databunch to get data from folders where each folder will be treated as a single class
# valid pct = split for training and validation
# bs = batch_size
# ds_tfms are transformation which will be used on image_data by deafult we use "get_transforma()" which by
# deafult does horizontal flip and other transformtaions like rotation, lighting, etc.


databunch = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
    ds_tfms=get_transforms(), size=448, bs = 4, num_workers=4).normalize(imagenet_stats)


# to avoid horizontal flip and rotation uncomment the 2 lines below

# databunch = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
#     ds_tfms=get_transforms(do_flip=False, max_rotate=0), size=size, bs = bs, num_workers=4).normalize(imagenet_stats)

In [13]:
# to know more about the databunch you have created
# print(data.classes, data.c)
# data.show_batch(10, figsize=(10,7))

In [14]:
# to find the best learning rate for training uncomment the last two lines(it's not necessary just use 1e-3 for lr)
# learn.lr_find()
# learn.recorder.plot()

In [15]:
# here databunch are data which is defined above "models.resnet34" is architecture
# to choose other architectures type "models." and then press "TAB"
# to use metrics other than accuracy go to the following link [fastai_metrics](https://docs.fast.ai/metrics.html#Training-metrics) 
# you can write your own function for metrics(it's pretty easy)

learn = cnn_learner(databunch, models.densenet201, metrics=accuracy)   

In [ ]:
# If you want to use efficient-net in fastai then run this cell:
# model = EfficientNet.from_name('efficientnet-b1')
# model._fc = nn.Linear(1280, data.c)
# learn = Learner(data, model, metrics=accuracy)

In [16]:
# to free_up some space 
# torch.cuda.empty_cache() 
# learn.unfreeze()

In [ ]:
# Model can also be trained on multiple GPUs using
# learn.model = torch.nn.DataParallel(learn.model, device_ids=[0,1])

In [17]:
# to start training run this cell, first argument is epoch and second one is learning rate 
# to train different layer groups with different learning rates use "slice(1e-6, 1e-4)"


# learn.fit_one_cycle(10, 1e-3)


# in the above case the model will not be saved instead it will create a "tmp.pth" file for model and it will be saved
# when we call learn.save
# to save the best fit model based on the accuracy during training without worrying about overfittng
# uncomment the last line
# here monitor is the judge to save model it can be changed to training loss or validation loss and best is the name for model

learn.fit_one_cycle(10, 1e-3, callbacks=[SaveModelCallback(learn, every='improvement', monitor='accuracy', name='best')])



epoch,train_loss,valid_loss,accuracy,time
0,1.050130,0.555015,0.802632,00:19
1,1.068513,0.655165,0.723684,00:13
2,0.980125,0.951555,0.776316,00:13
3,0.828842,0.622180,0.750000,00:13
4,0.787156,0.546933,0.789474,00:13
5,0.779973,0.807844,0.684211,00:13
6,0.745200,0.561863,0.684211,00:13
7,0.520234,0.473378,0.802632,00:13
8,0.449294,0.467562,0.828947,00:13
9,0.502472,0.445076,0.842105,00:13


Better model found at epoch 0 with accuracy value: 0.8026315569877625.
Better model found at epoch 8 with accuracy value: 0.8289473652839661.
Better model found at epoch 9 with accuracy value: 0.8421052694320679.


In [ ]:
#     to save and load the model uncomment the below two lines
#     learn.save("name of model.pth")

#     if you are using best fit callback in learner no need to save model just load the model with the saved name
#     learn.load("name of model.pth")

#     to save without optimizer data(it will have no effect on accuracy but you can't resume training from where you left)
#     it will reduce size of model
#     learn.save("name of model.pth", with_opt=False)

#     to save using plane torch
#     torch.save(learn.model, "./final.pth")

In [16]:
# to check the results
# learn.show_results()

In [18]:
# to export and save the whole learner and load for next time
learn.export("model.pkl")

# and to load
# load_learner(path, 'name .pkl')

In [24]:
# to reduce the size of model further the model can be trained in fp16 and to do this
# just convert the learner i.e, "learn.to_fp16" before training


In [27]:
# to only save state_dict of model
# torch.save({'state_dict': learn.model.state_dict()}, './name .pth')